In [3]:
using Distributed
using Statistics
using BenchmarkTools
using LinearAlgebra
using Distributed
using ProgressMeter
using FFTW
using MLDatasets
using Images
using Interpolations
using HDF5
using IterTools
push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
using DHC_2DUtils

In [4]:
M = 2*360
angle_array = []
for i = 0:2π/M:2π
    append!(angle_array,i)
end

In [5]:
addprocs(7);

In [6]:
L = 8
rand_mat = rand(16,16)
filter_hash = fink_filter_hash(1,L,nx=16,wd=2)

Dict{Any,Any} with 9 entries:
  "theta_value" => [0.0, 0.392699, 0.785398, 1.1781, 1.5708, 1.9635, 2.35619, 2…
  "pc"          => 1
  "filt_index"  => Array{CartesianIndex{2},1}[[CartesianIndex(2, 3), CartesianI…
  "filt_value"  => [[0.150094, 0.150094, 0.794909, 0.68921, 0.374019, 0.374019,…
  "npix"        => 16
  "psi_index"   => Int32[1 3 … 13 15; 2 4 … 14 16]
  "phi_index"   => 17
  "wd"          => 2
  "j_value"     => [1.0, 2.0]

In [8]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    train_x, train_y = MNIST.traindata()
    test_x, test_y   = MNIST.testdata()
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=3)
    
    lst = Array{Any}(undef, 0)
    for i = 1:60000
        push!(lst,train_x[:,:,i])
    end
    
    img_prep = lst[3];

    function mnist_pad(θ)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = img_prep'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(θ)
        image    = mnist_pad(θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
end

mnist_DHC_out = @showprogress pmap(mnist_DHC, angle_array)
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("../scratch_AKS/mnist_DHC_angles_1_1_wd3.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:12


In [9]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    train_x, train_y = MNIST.traindata()
    test_x, test_y   = MNIST.testdata()
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=1)
    
    lst = Array{Any}(undef, 0)
    for i = 1:60000
        push!(lst,train_x[:,:,i])
    end
    
    img_prep = lst[3];

    function mnist_pad(θ)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = img_prep'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(θ)
        image    = mnist_pad(θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
end

mnist_DHC_out = @showprogress pmap(mnist_DHC, angle_array)
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("../scratch_AKS/mnist_DHC_angles_1_1_wd1.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


In [11]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    train_x, train_y = MNIST.traindata()
    test_x, test_y   = MNIST.testdata()
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=1)
    filter_hash2 = fink_filter_hash(1,8,nx=128,wd=1,shift=true)
    
    lst = Array{Any}(undef, 0)
    for i = 1:60000
        push!(lst,train_x[:,:,i])
    end
    
    img_prep = lst[3];

    function mnist_pad(θ)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = img_prep'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(θ)
        image    = mnist_pad(θ)
        WST = DHC_compute(image, filter_hash, filter_hash2)
        return WST
    end
end

mnist_DHC_out = @showprogress pmap(mnist_DHC, angle_array)
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("../scratch_AKS/mnist_DHC_angles_1_1_wd1_shift.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


In [12]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    train_x, train_y = MNIST.traindata()
    test_x, test_y   = MNIST.testdata()
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=3)
    filter_hash2 = fink_filter_hash(1,8,nx=128,wd=3,shift=true)
    
    lst = Array{Any}(undef, 0)
    for i = 1:60000
        push!(lst,train_x[:,:,i])
    end
    
    img_prep = lst[3];

    function mnist_pad(θ)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = img_prep'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(θ)
        image    = mnist_pad(θ)
        WST = DHC_compute(image, filter_hash, filter_hash2)
        return WST
    end
end

mnist_DHC_out = @showprogress pmap(mnist_DHC, angle_array)
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("../scratch_AKS/mnist_DHC_angles_1_1_wd3_shift.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09


In [13]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    train_x, train_y = MNIST.traindata()
    test_x, test_y   = MNIST.testdata()
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=2)
    filter_hash2 = fink_filter_hash(1,8,nx=128,wd=2,shift=true)
    
    lst = Array{Any}(undef, 0)
    for i = 1:60000
        push!(lst,train_x[:,:,i])
    end
    
    img_prep = lst[3];

    function mnist_pad(θ)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = img_prep'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(θ)
        image    = mnist_pad(θ)
        WST = DHC_compute(image, filter_hash, filter_hash2)
        return WST
    end
end

mnist_DHC_out = @showprogress pmap(mnist_DHC, angle_array)
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("../scratch_AKS/mnist_DHC_angles_1_1_wd2_shift.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
